# Weather Agent

In this notebook, demonstrate how to create and use a `meteorologist`
agent that retrieves real-time weather data using a "get_weather" function.
We will define the agent's capabilities and see how it processes user queries to
provide accurate and current weather information for a given location.

In [1]:
%useLatestDescriptors
@file:DependsOn("ai.ancf.lmos:arc-langchain4j-client:0.120.0")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.0.0-beta1")

In this cell, we configure our environment with two key variables:
* `openAiApiKey` for [OpenAI services](https://platform.openai.com/docs/api-reference/authentication) (required to interact with GPT-based models)
* `weatherApiKey` for [WeatherAPI](https://www.weatherapi.com/) (required to fetch real-time weather data)

Make sure to replace the placeholder values (e.g., `"YOUR-OPENAI-API-KEY"`) with your actual keys before running the notebook.

In [2]:
import ai.ancf.lmos.arc.agents.llm.ChatCompleter
import ai.ancf.lmos.arc.agents.llm.ChatCompletionSettings
import ai.ancf.lmos.arc.client.langchain4j.LangChainClient
import ai.ancf.lmos.arc.client.langchain4j.LangChainConfig
import dev.langchain4j.model.openai.OpenAiChatModel

val openAiApiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR-OPENAI-API-KEY"
val weatherApiKey = System.getenv("WEATHER_API_KEY") ?: "YOUR-WEATHERAPI-API-KEY"

val chatCompleterProvider: (String?) -> ChatCompleter = {
    LangChainClient(
        languageModel = LangChainConfig(
            modelName = "gpt-4",
            url = null,
            apiKey = openAiApiKey,
            accessKeyId = null,
            secretAccessKey = null,
        ), clientBuilder = { config, _ ->
            OpenAiChatModel.builder()
                .modelName(config.modelName)
                .apiKey(config.apiKey)
                .build()
        }
    )
}

Here, we define a “meteorologist” agent using DSLAgents.
This agent has a prompt describing its purpose (to provide weather data) and a custom function,
`get_weather`, which fetches real-time weather information.
It leverages our defined `weatherApiKey` to make an HTTP request to WeatherAPI.
The agent can then respond with up-to-date weather details for any location requested by the user.

In [3]:
import ai.ancf.lmos.arc.agents.DSLAgents

val agentName = "meteorologist"
val agentBuilder = DSLAgents.init(chatCompleterProvider).apply {
    define {
        agent {
            name = "meteorologist"
            description = "A meteorologist can tell you the current weather in a given location."
            prompt {
                """
                    You are a professional weather service.
                    You provide weather data to you users.
                    You have access to real-time weather data with the get_weather function.
                    Use 'unknown' if the location is not provided.
                    Always state the location used in the response.

                    # Instructions
                    - Use the get_weather function to get the weather data.
                    - Use multiple function calls if more locations are specified.
                """
            }
            tools = listOf("get_weather")
        }
    }

    defineFunctions {
        function(
            name = "get_weather",
            description = "Get the weather for a given location",
            params = types(string("location", "the location")),
        ) { (location) ->
            val apiUrl = "http://api.weatherapi.com/v1/current.json"
            httpGet("$apiUrl?key=$weatherApiKey&q=$location")
        }
    }
}

In [4]:
val agents = agentBuilder.getAgents()
agents

[ChatAgent(name='meteorologist', description='A meteorologist can tell you the current weather in a given location.')]

Let's retrieve the weather for Berlin by sending the user's question to our "meteorologist" agent.
We define a new conversation containing the user's message and then execute the conversation, getting the weather data in the agent's response.

In [5]:
import ai.ancf.lmos.arc.agents.ChatAgent
import ai.ancf.lmos.arc.agents.User
import ai.ancf.lmos.arc.agents.conversation.Conversation
import ai.ancf.lmos.arc.agents.conversation.UserMessage
import ai.ancf.lmos.arc.agents.getAgentByName
import ai.ancf.lmos.arc.core.getOrNull
import kotlinx.coroutines.runBlocking

val agent = agentBuilder.getAgentByName("meteorologist") as ChatAgent? ?: error("Agent not found!")
val conversation = Conversation(User("userOrClientId")) + UserMessage("What is the weather like in Berlin?")
runBlocking {
    agent.execute(conversation).getOrNull()?.transcript?.getOrNull(1)?.content
}

The current weather in Berlin, Germany is overcast with a temperature of 3.3°C (37.9°F). The wind is coming from the East at a speed of 18.7 kph (11.6 mph). The air pressure is 1033.0 mb. The humidity level is at 75%. The visibility is 10.0 km (6.0 miles). The UV index is 0.0.